<table>
    <tr>
      <td>Introducción a
      </td>
      <td>
      <img src="https://media.licdn.com/dms/image/D5612AQF7GSp3l4pztQ/article-cover_image-shrink_720_1280/0/1686548640655?e=1715817600&v=beta&t=WQzv1EMkEEwZ0QZ0PF1anRKIHCl5BBH_YPZHdDQsWPM"  width=150/>
      </td>
     </tr>
</table>
Rafa Caballero




# Tratamiento de nulos



### Índice
[Introducción](#Introducción)<br>
[Contando Nulos](#Contando)<br>
[Nulos graficamente](#Graficamente)<br>
[No nulos, pero casi](#Nonulos)<br>
[¿Qué hacer?](#Que)<br>
[Tipos](#Tipos)<br>
  &nbsp;&nbsp;&nbsp;&nbsp;  [MD versus MI en variables categóricas](#Categorical)<br>
  &nbsp;&nbsp;&nbsp;&nbsp;  [MD versus MI en variables ratio](#Ratio)<br>
[Bibliografía](#Bibliografía)<br>

<a name="Intro"></a>
## Introducción

Los valores nulos llamados en inglés *missing* son *huecos* o valores que faltan en nuestro dataset. Puedne impedir que se hagan algunas operaciones o introducir sesgo en otras. Entender por qué surgen y buscar formas de corregirlos es importante para lograr datos de calidad.

<a name="Contando"></a>
## Contando nulos

Los valores nulos se representan mediante la constante NaN que se puede obtener como

`pd.NA`, `float('nan')`, `math.nan`, or `np.nan`. También el valor `None` se cuenta en Pandas como un valor NaN.




In [ ]:
import numpy as np
import pandas as pd
import math
df = pd.DataFrame({'a': [1, np.nan, 3, math.nan, float('nan'), None],
                   'b': [1, pd.NA, 3, 4, 6, None],
                   'c': [1, 2, 3, 4, 6, np.nan]})
df

In [ ]:
df.info()

Podemos ver los nulos por columnas con el método `isna`

In [ ]:
df.isna().sum()

Para ver los totales por fila:

In [ ]:
df.isna().sum(axis=1)

Columnas con algún nulo

In [ ]:
(df.isna().sum() > 0).sum()

filas com algún nulo:

In [ ]:
(df.isna().sum(axis=1) > 0).sum()

Columnas con todo nulos

In [ ]:
(df.isna().sum() == df.shape[0]).sum()

Filas con todo nulos:

In [ ]:
(df.isna().sum(axis=1) == df.shape[1]).sum()

Y el total absoluto:

In [ ]:
total = df.isna().sum().sum()
total

También es interesante ver la proporción de nulos:

In [ ]:
round(100*total/(df.shape[0]*df.shape[1]),3)

<a name="Graficamente"></a>
## Nulos gráficamente
Sobre todo en el caso de dataframes con gran cantidad de datos utilizar una visualización adecuada puede ayudar a entender el origen de los nulos

In [ ]:
pip install missingno

In [ ]:
import seaborn as sns
import pandas as pd
import missingno as msno
%matplotlib inline

titanic = sns.load_dataset("titanic")
titanic.info()

In [ ]:
msno.bar(titanic)

Otra forma de ver lo mismo

In [ ]:
msno.matrix(titanic)

ESto nos da mucha información y muy útil:

* Parece que solo age y deck tienen nulos.

* Aparte hay 2 filas que tienen nulos también en `embarked`, `embark_town` (se podrían eliminar?)

El método `heatmap` nos ayudará a relacionar columnas con nulos:

In [ ]:
msno.heatmap(titanic)

Vemos que siempre que embarked es missing también lo es embark_town.

<a name="Nonulos"></a>
## No nulos, pero casi
En ocasiones hay valores que no son nulos pero son "señales" que indican falta de información. Valores como "99" o valores negativos en columnas que deben tener valores no negativos. Lo que debemos hacer es convertir estos valores en NaN

In [ ]:
url="https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/NATOships.csv"
df2 = pd.read_csv(url)
df2

Queremos reemplazar los valores "-" por NaN

In [ ]:
df2["status"] = df2["status"].replace("-",pd.NA)
df2

In [ ]:
msno.matrix(df2)

In [ ]:
msno.heatmap(df2)

<a name="Que"></a>
## ¿Qué hacer?

* Si hay columnas con muy pocos datos válidos y que no resultan imprescindibles se pueden borrar

* Si hay muy pocas filas con un valor nulo (representan un tanto por cierto muy pequeño, menor del 1%) se pueden eliminar

* Si el número de nulos es muy alto una posibilidad es *imputar* el valor. Las formas típicas de imputar:
    - La mediana
    - La media
    - La moda (también vale para variables nominales)

In [ ]:
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/contaminacionFinal.csv"
df_conta = pd.read_csv(url)
msno.matrix(df_conta)


En la columna 6 queremos reemplazar el valor por el valor medio; para eso utilizaremos un [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

In [ ]:

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer( strategy='mean') #for median imputation replace 'mean' with 'median'
imp_mean.fit(df_conta[["6"]])
df_imputed = df_conta.copy()
df_imputed[["6"]]  = imp_mean.transform(df_conta[["6"]])
msno.matrix(df_imputed)

<a name="Tipos"></a>
## Tipos de nulos

En [1976](https://www.jstor.org/stable/2335739) Donald B. Rubin distinguió 3 tipos de valores nulos:

* Si los datos perdidos se encuentran totalmente al azar en una columna y no depende del valor del resto de la fila, decimos que es  *missing completely at random* (MCAR). Nosotros les vamos a llamar Missing Independientes (MI).

* Si el valor perdido está asociado a ciertos valores del resto de la fila tenemos valores *missing at random* (MAR), o Missing Dependientes (MD)

* En otro caso hablamos de *missing not at random* (MNAR): hay otras causas que no conocemos, no los vamos a considerar



<a name="Categorical"></a>
### MI versus MD en variables categóricas.

Podemos detectar si una columna depende de otra como (y por tanto es MAR y no MCAR) utilizando el test $\mathcal{X}^2$.
Empecemos por generar datos MI

In [ ]:
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/contaminacionFinal.csv"
df_conta = pd.read_csv(url)
df_conta

Nos inventamos una columna nueva que va a ser nula dependiendo del valor de un dado

In [ ]:
import numpy as np
df_conta["15"] = 0 # para crearlo
df_conta["dado"] = 0 # para crearlo

df_conta["dado"]  = np.random.choice([1, 2, 3, 4, 5, 6], len(df_conta), p=[1/6, 1/6, 1/6, 1/6, 1/6, 1/6])

# nos quedamos con las filas en las que el dado tiene un 6
index = df_conta[df_conta["dado"]==6].index

# le ponemos nulos
df_conta.loc[index,"15"] = pd.NA
df_conta

Para aplicar el test convertimos la columna que queremos examinar, la "15", en una nueva que vale 1 si es nulo y 0  en otro caso

In [ ]:
df_conta["15_missing"] = 0
indice_nulos = df_conta[df_conta['15'].isnull()].index
df_conta.loc[indice_nulos, "15_missing"] = 1
df_conta

Veamos primero cómo se relaciona con una columna nominal

In [ ]:
contingencias=pd.crosstab(index=df_conta.ANO,columns=df_conta["15_missing"])
contingencias

In [ ]:
from scipy.stats import chi2_contingency

# este es el test
ChiSqResult = chi2_contingency(contingencias)
ChiSqResult

Como $H_0$ = No hay correlación entre las variables, tenemos que como p>0.05 no podemos descartar no haya correlación, en principio asumimos MI. Miremos otra variable:

In [ ]:
df_conta["9_missing"] = 0
indice_nulos = df_conta[df_conta['9'].isnull()].index
df_conta.loc[indice_nulos, "9_missing"] = 1
contingencias=pd.crosstab(index=df_conta.ANO,columns=df_conta["9_missing"])
ChiSqResult = chi2_contingency(contingencias)
ChiSqResult

Sale que sí hay correlación con el año; estamos en el caso de MD. Veamos si podemos ver esta relación gráficamente:

In [ ]:
sns.histplot(data=df_conta, hue="9_missing",x="ANO",stat="count", multiple="stack")

Cabría pensar en si merece la pena tratar aparte el año 2019

<a name="Ratio"></a>
### MI  versus MD en variables ratio

En este caso usaremos las correlaciones


In [ ]:
df_conta

In [ ]:
df_conta.iloc[:,4:-3]

In [ ]:
df_conta_num = df_conta.iloc[:,4:-3].copy()

for c in df_conta_num:
    indice_nulos = df_conta_num[df_conta_num[c].isnull()].index
    df_conta_num[c] = 0
    df_conta_num.loc[indice_nulos, c] = 1
df_conta_num


In [ ]:
# quitamos las columnas que no tienen unos
tiene_nulos = df_conta_num.sum()>0
tiene_nulos

In [ ]:
df_conta_num = df_conta_num.iloc[:,tiene_nulos.values]
df_conta_num

In [ ]:
import matplotlib.pyplot as plt
corr = df_conta_num.corr()
plt.figure(figsize=(11,8))
sns.heatmap(corr, cmap="Greens",annot=True)
plt.show()

Por tanto las columnas 9 y 10 parecen tener nulos justamente en las mismas posiciones y hablamos de MD, quizás tengan que estudiarse aparte

<a name="Bibliografía"></a>
## Bibliografía

[Visualización](https://towardsdatascience.com/visualizing-missing-values-in-python-is-shockingly-easy-56ed5bc2e7ea) con el ejemplo del titanic que hemos mostrado

[MCAR, MAR, MNAR](https://stefvanbuuren.name/fimd/sec-MCAR.html)

[El test $\mathcal{X}^2$ para distinguir MCAR de MAR](https://www.kaggle.com/code/yassirarezki/handling-missing-data-mcar-mar-and-mnar-part-i/notebook)